In [194]:
import os
import pandas as pd
import json
import requests
from flatten_json import flatten
from datetime import datetime, timezone

In [87]:
def generate_data (
      start = 1,
      end = 2,
      api_key = os.getenv('API_OPENWEATHER')):
      base_url = f"https://history.openweathermap.org/data/2.5/history/city?q=Bogor,ID&type=day&start={start}&cnt={end}&appid={api_key}"
      response = requests.get(base_url)
      x = response.json()
      return x

from : 1656633600
to : 1664668740

In [204]:
json_raw = []
for i in range(1656633600, 1665100800, 604800):
    x = generate_data(start=i, end=1665100800)
    json_raw.append(x)

In [205]:
json_raw_1 = []
for i in range(len(json_raw)):
    x = json_raw[i]['list']
    json_raw_1.append(x)


In [206]:
temp = []
for i in range(len(json_raw_1)):
    temporary = pd.json_normalize(json_raw_1[i],
                                record_path=['weather'],
                                meta = ['dt',
                                ['main', 'temp'],
                                ['main', 'feels_like'],
                                ['main', 'humidity'],
                                ['main', 'temp_min'],
                                ['main', 'temp_max'],
                                ['wind', 'speed'],
                                ['wind', 'deg'],
                                ['wind', 'gust'],
                                ['clouds', 'all'],
                                ['rain', '1h']], errors='ignore')
    temp.append(temporary)

df = pd.concat(temp, axis=0, ignore_index=True)

df.sample(10)
    

,id,main,description,icon,dt,main.temp,main.feels_like,main.humidity,main.temp_min,main.temp_max,wind.speed,wind.deg,wind.gust,clouds.all,rain.1h
2353,501,Rain,moderate rain,10n,1665057600,298.06,298.98,91,298.06,298.06,0.68,79,0.98,91,2.18
384,804,Clouds,overcast clouds,04n,1658008800,296.74,297.45,88,296.74,296.74,1.24,188,1.27,96,NaN
1391,804,Clouds,overcast clouds,04n,1661612400,294.71,295.35,93,294.71,296.18,1.01,149,1.07,100,NaN
1859,804,Clouds,overcast clouds,04d,1663286400,299.2,299.2,86,298.41,299.2,1.52,180,2.12,96,NaN
727,802,Clouds,scattered clouds,03d,1659236400,304.2,309.71,66,303.96,304.2,1.83,18,1.89,38,NaN
1638,802,Clouds,scattered clouds,03n,1662498000,294.16,294.88,98,294.16,298.09,1.08,183,1.06,42,NaN
1481,804,Clouds,overcast clouds,04d,1661936400,305.73,309.16,52,302.85,305.82,1.84,31,2.48,90,NaN
2290,804,Clouds,overcast clouds,04n,1664830800,298.65,299.61,90,296.74,298.65,1.84,182,2.47,100,NaN
894,804,Clouds,overcast clouds,04d,1659834000,296.93,297.61,86,296.93,300.63,1.32,169,1.56,87,NaN
2313,804,Clouds,overcast clouds,04n,1664913600,298.09,299.04,92,296.74,298.09,1.53,207,2.03,100,NaN


In [211]:
df['dt'] = pd.to_datetime(df['dt'], unit='s', utc=True)

In [212]:
df

,id,main,description,icon,dt,main.temp,main.feels_like,main.humidity,main.temp_min,main.temp_max,wind.speed,wind.deg,wind.gust,clouds.all,rain.1h
0,803,Clouds,broken clouds,04d,2022-07-01 00:00:00+00:00,297.3,297.99,85,297.3,297.3,0.84,158,0.89,76,NaN
1,804,Clouds,overcast clouds,04d,2022-07-01 01:00:00+00:00,299.52,299.52,77,299.52,299.52,0.23,148,0.5,100,NaN
2,804,Clouds,overcast clouds,04d,2022-07-01 02:00:00+00:00,299.43,299.43,71,299.43,299.43,1.05,29,1.21,88,NaN
3,803,Clouds,broken clouds,04d,2022-07-01 03:00:00+00:00,300.65,302.36,65,300.65,300.65,1.75,15,1.75,83,NaN
4,803,Clouds,broken clouds,04d,2022-07-01 04:00:00+00:00,301.49,303.11,60,301.49,301.49,1.81,16,1.92,81,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2361,804,Clouds,overcast clouds,04n,2022-10-06 20:00:00+00:00,298.09,298.99,90,296.74,298.09,1.22,225,1.39,100,NaN
2362,804,Clouds,overcast clouds,04n,2022-10-06 21:00:00+00:00,298.09,298.96,89,296.18,298.09,1.58,255,1.58,100,NaN
2363,804,Clouds,overcast clouds,04n,2022-10-06 22:00:00+00:00,296.18,296.86,89,296.18,296.18,1.48,240,1.5,100,NaN
2364,804,Clouds,overcast clouds,04d,2022-10-06 23:00:00+00:00,296.18,296.86,89,296.18,296.18,1.25,230,1.34,100,NaN


In [213]:
df.to_parquet('../data/interim/weather_data_final.parquet',
              index=False)